In [46]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()

url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [47]:
import pandas as pd

In [48]:
DB_TABLE_NAME = "Inventory"


def extract_inventory_data():
    response = supabase.table(DB_TABLE_NAME).select("*").execute()
    if response and hasattr(response, 'data'):
        return response.data
    return response or "No data found"


In [49]:
data = extract_inventory_data()

In [50]:
data[0].keys()

dict_keys(['created_at', 'item_name', 'quantity', 'description'])

In [51]:
import csv

In [52]:
with open("output.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print("✅ CSV file saved as output.csv")

✅ CSV file saved as output.csv


In [53]:
response = supabase.table(DB_TABLE_NAME).upsert({"item_name": "Sample", "quantity": 15, "description": "Sample"}).execute()

In [54]:
response.data

[{'created_at': '2025-10-12T17:03:37.218016+00:00',
  'item_name': 'Sample',
  'quantity': 15,
  'description': 'Sample'}]

In [55]:
from pydantic import BaseModel, Field
from typing import Optional

In [56]:
class UpsertState(BaseModel):
    item_name: str = Field(..., description="Name of the inventory item")
    quantity: int = Field(..., description="Quantity of the inventory item")
    description: Optional[str] = Field(None, description="Description of the inventory item")

In [57]:
# # Inventory Management Multi-Agent System

# This notebook demonstrates the multi-agent inventory management system that we've built. The system uses LangGraph to coordinate between multiple specialized agents that work together to manage inventory data.

# ## System Components:
# 1. Task Classifier - Determines what the user wants to do
# 2. QA Agent - Answers questions about inventory
# 3. Upsert Agent - Adds or updates inventory items
# 4. Delete Agent - Removes items from inventory
# 5. CSV Export - Exports current inventory to CSV

In [58]:
# Import necessary modules
import os
import sys
from colorama import Fore, init

# Add current directory to path if needed
sys.path.append(os.getcwd())

from agent import (
    initialize_agent_state,
    classify_task,
    run_qa_agent,
    run_upsert_agent,
    run_delete_agent,
    export_data_to_csv
)
from graph import build_workflow_graph
from tools import extract_inventory_data

# Initialize colorama
init(autoreset=True)

# Check if all modules are loaded correctly
print("✅ All modules loaded successfully")

/home/aman/miniconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


✅ All modules loaded successfully


In [ ]:
# Import necessary modules
import os
import sys
from colorama import Fore, init

# Add current directory to path if needed
sys.path.append(os.getcwd())

from agent import (
    initialize_agent_state,
    classify_task,
    run_qa_agent,
    run_upsert_agent,
    run_delete_agent,
    export_data_to_csv
)
from graph import build_workflow_graph
from tools import extract_inventory_data

# Initialize colorama
init(autoreset=True)

# Check if all modules are loaded correctly
print("✅ All modules loaded successfully")

In [59]:
# Build the workflow graph
workflow = build_workflow_graph(
    classify_task_fn=classify_task,
    run_qa_agent_fn=run_qa_agent,
    run_upsert_agent_fn=run_upsert_agent,
    run_delete_agent_fn=run_delete_agent,
    export_csv_fn=export_data_to_csv
)

print("✅ Workflow graph built successfully")

/home/aman/miniconda3/envs/langchain/lib/python3.11/site-packages/langgraph/graph/state.py:103: UserWarning: Invalid state_schema: inventory_agent. Expected a type or Annotated[type, reducer]. Please provide a valid schema to ensure correct updates.
 See: https://langchain-ai.github.io/langgraph/reference/graphs/#stategraph
  warnings.warn(


RuntimeError: 

In [ ]:
# Display current inventory
def display_inventory():
    """Print the current inventory."""
    inventory = extract_inventory_data()
    
    if not inventory:
        print(Fore.YELLOW + "Inventory is empty.")
        return inventory
    
    print(Fore.YELLOW + f"Current Inventory ({len(inventory)} items):")
    print(Fore.YELLOW + "-" * 80)
    print(Fore.YELLOW + f"{'ID':<5} {'Item Name':<20} {'Quantity':<10} {'Description':<40}")
    print(Fore.YELLOW + "-" * 80)
    
    for item in inventory:
        item_id = item.get("id", "N/A")
        item_name = item.get("item_name", "N/A")
        quantity = item.get("quantity", 0)
        description = item.get("description", "")
        print(Fore.WHITE + f"{item_id:<5} {item_name:<20} {quantity:<10} {description:<40}")
    
    print(Fore.YELLOW + "-" * 80)
    return inventory

# Display current inventory
inventory_data = display_inventory()

## Testing the Agents

Let's test our multi-agent system with different types of queries:
1. Fetch query (QA Agent)
2. Upsert query (Update/Insert Agent)
3. Delete query (Delete Agent)

In [ ]:
# Test 1: QA Agent - Fetching inventory data
query = "Show me all inventory items"
print("User Query:", query)
print("\nProcessing...\n")
result = workflow.invoke({"message": query})
print("\nAgent Response:")
print(result["response"])

In [ ]:
# Test 2: Upsert Agent - Adding or updating inventory items
query = "Add new item called 'Laptop' with quantity 10 and description 'Dell XPS 13'"
print("User Query:", query)
print("\nProcessing...\n")
result = workflow.invoke({"message": query})
print("\nAgent Response:")
print(result["response"])

# Display updated inventory after upsert
print("\nUpdated Inventory:")
inventory_items = tools.extract_inventory_data()
display(pd.DataFrame(inventory_items))

In [ ]:
# Test 3: Delete Agent - Removing inventory items
query = "Delete the Laptop item from inventory"
print("User Query:", query)
print("\nProcessing...\n")
result = workflow.invoke({"message": query})
print("\nAgent Response:")
print(result["response"])

# Display updated inventory after deletion
print("\nUpdated Inventory after deletion:")
inventory_items = tools.extract_inventory_data()
display(pd.DataFrame(inventory_items))

## Advanced Testing

Let's test more complex queries and edge cases:

In [ ]:
# Test 4: Multiple items addition
query = "Add the following items: 5 Monitors with description 'Dell 27-inch 4K', 3 Keyboards with description 'Mechanical RGB', and 8 Mouse devices with description 'Wireless Optical'"
print("User Query:", query)
print("\nProcessing...\n")
result = workflow.invoke({"message": query})
print("\nAgent Response:")
print(result["response"])

# Display updated inventory
print("\nUpdated Inventory:")
inventory_items = tools.extract_inventory_data()
display(pd.DataFrame(inventory_items))

In [ ]:
# Test 5: Updating existing items
query = "Update the quantity of Monitors to 10 and change the description to 'Dell 27-inch 4K UHD'"
print("User Query:", query)
print("\nProcessing...\n")
result = workflow.invoke({"message": query})
print("\nAgent Response:")
print(result["response"])

# Display updated inventory
print("\nUpdated Inventory:")
inventory_items = tools.extract_inventory_data()
display(pd.DataFrame(inventory_items))

In [ ]:
# Test 6: Export to CSV
query = "Export all inventory data to CSV"
print("User Query:", query)
print("\nProcessing...\n")
result = workflow.invoke({"message": query})
print("\nAgent Response:")
print(result["response"])

# Check if the CSV file exists and display its contents
import os
if os.path.exists("output.csv"):
    print("\nCSV Export Contents:")
    csv_df = pd.read_csv("output.csv")
    display(csv_df)
else:
    print("CSV file not found")

## Visualizing the Agent Workflow

Let's visualize the LangGraph workflow to better understand the agent interactions:

In [ ]:
# Import necessary visualization libraries
try:
    from IPython.display import Image
    from langchain_core.graphs import graph
    
    # Generate the workflow graph visualization
    workflow_graph = workflow.get_graph()
    
    # Save the graph to a temporary file and display it
    graph_image = graph.visualize(workflow_graph)
    display(Image(graph_image))
except Exception as e:
    print(f"Unable to visualize graph: {e}")
    print("If visualization is not working, make sure graphviz is installed.")
    print("You can install it using: pip install graphviz")
    
    # Display textual representation instead
    print("\nWorkflow Structure (Textual Representation):")
    print("1. Task Classifier -> Determines query type")
    print("2. Routes to one of the following agents:")
    print("   - QA Agent: Fetch inventory data")
    print("   - Upsert Agent: Add/Update inventory items")
    print("   - Delete Agent: Remove inventory items")
    print("3. Returns response to user")

## Conclusion

This notebook demonstrates a complete multi-agent inventory management system with LangGraph. The system includes:

1. **Task Classification Agent**: Analyzes user queries and routes them to the appropriate specialized agent
2. **QA Agent**: Retrieves inventory information and answers queries about the data
3. **Upsert Agent**: Adds new items or updates existing ones in the inventory
4. **Delete Agent**: Removes items from the inventory

The system leverages LangGraph for agent orchestration, Pydantic for structured data validation, and SQL database for persistent storage. The workflow processes natural language queries and performs the appropriate database operations based on the intent of the query.

### Key Components:
- **prompt.py**: Contains all agent prompts
- **tools.py**: Provides database interaction functions
- **agent.py**: Defines agent states and functions
- **graph.py**: Sets up the LangGraph workflow
- **main.py**: Application entry point

### Future Enhancements:
- Add more specialized agents (e.g., for inventory analysis, alerting on low stock)
- Implement authentication and authorization
- Add batch operations for efficiency
- Create a web interface for better usability